In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
12,application_1584073442397_0013,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-12>

In [ ]:
sc.list_packages()

In [82]:
from pyspark.sql.types import *
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import warnings; warnings.simplefilter('ignore')
import datetime

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [83]:
# config spark to build context
spark=SparkSession.builder.appName("NYC.TLC-Green-Car-Data-ETL").getOrCreate()
spark.conf.set("spark.sql.session.timeZone", "America/New_York")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [84]:
#read in our raw dataset
df = spark.read.option('header', 'true') \
    .option('mode', 'FAILFAST') \
    .option('timeStampFormat', 'yyyy-MM-dd HH:mm:ss') \
    .option('columnNameOfCorruptRecord', 'error') \
    .csv('s3://data-etl-o-original-raw/green/*.csv') \
    .filter(year('lpep_pickup_datetime') == 2019) \
    .withColumn('duration', f.unix_timestamp('lpep_dropoff_datetime') - f.unix_timestamp('lpep_pickup_datetime')) \
    .withColumn('minute_rate',f.when(f.col('total_amount') <= 0.001,0).otherwise(f.col('total_amount')/f.col('duration') * 60)) \
    .withColumn('average_speed',f.when(f.col('trip_distance') < 0.001,0).otherwise(f.col('trip_distance')/f.col('duration') * 60 * 60)) \
    .drop('store_and_fwd_flag') \
    .drop('ehail_fee') \
    .drop('congestion_surcharge')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [85]:
dfcache = df.cache()

dfcache

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[VendorID: string, lpep_pickup_datetime: string, lpep_dropoff_datetime: string, RatecodeID: string, PULocationID: string, DOLocationID: string, passenger_count: string, trip_distance: string, fare_amount: string, extra: string, mta_tax: string, tip_amount: string, tolls_amount: string, improvement_surcharge: string, total_amount: string, payment_type: string, trip_type: string, duration: bigint, minute_rate: double, average_speed: double]

In [9]:
dfcache.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime', 'RatecodeID', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'payment_type', 'trip_type', 'duration', 'minute_rate', 'average_speed']

In [86]:
## check schema structure
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- VendorID: string (nullable = true)
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- lpep_dropoff_datetime: string (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- trip_type: string (nullable = true)
 |-- duration: long (nullable = true)
 |-- minute_rate: double (nullable = true)
 |-- average_speed: double (nullable = true)

In [67]:
# do some test
tmp = df.select(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime', 'duration','total_amount','trip_distance','minute_rate','average_speed'])
tmp.show(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+---------------------+--------+------------+-------------+--------------------+-------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|duration|total_amount|trip_distance|         minute_rate|      average_speed|
+--------+--------------------+---------------------+--------+------------+-------------+--------------------+-------------------+
|       2| 2019-01-01 00:10:16|  2019-01-01 00:16:32|     376|         7.3|          .86|  1.1648936170212767|   8.23404255319149|
|       2| 2019-01-01 00:27:11|  2019-01-01 00:31:38|     267|         5.8|          .66|   1.303370786516854|  8.898876404494382|
|       2| 2019-01-01 00:46:20|  2019-01-01 01:04:54|    1114|       19.71|         2.68|  1.0615798922800719|  8.660682226211849|
|       2| 2019-01-01 00:19:06|  2019-01-01 00:39:43|    1237|        19.3|         4.53|  0.9361358124494745| 13.183508488278092|
|       2| 2019-01-01 00:12:35|  2019-01-01 00:19:09|     394|         7.8|        

In [89]:
dfcache.count(),df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(5929810, 5779303)

In [10]:
df.columns,dfcache.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime', 'RatecodeID', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'payment_type', 'trip_type', 'duration', 'minute_rate', 'average_speed']

In [90]:
## fix colums with regular names
for c in df.columns:
    df = df.withColumnRenamed(c, c.replace(' ', '_'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [87]:
## polish data

## remove data which is tooooo small and ilegal value
df = df.filter(df['trip_distance'] >= 0.001)

## round minute_rate and average_speed to scale 3 

df = df.withColumn('minute_rate', round(df['minute_rate'],4))

df = df.withColumn('average_speed', round(df['average_speed'],4))

# rename 'VendorID'
# df = df.withColumnRenamed('VendorID', 'vendor_id')

df.show(10)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+---------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+---------+--------+-----------+-------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|payment_type|trip_type|duration|minute_rate|average_speed|
+--------+--------------------+---------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+---------+--------+-----------+-------------+
|       2| 2019-01-01 00:10:16|  2019-01-01 00:16:32|         1|          97|          49|              2|          .86|          6|  0.5|    0.5|         0|           0|                  0.3|         7.3|   

In [12]:
tmp = df.select(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime', 'duration','total_amount','trip_distance','minute_rate','average_speed'])
tmp.show(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+---------------------+--------+------------+-------------+-----------+-------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|duration|total_amount|trip_distance|minute_rate|average_speed|
+--------+--------------------+---------------------+--------+------------+-------------+-----------+-------------+
|       2| 2019-01-01 00:10:16|  2019-01-01 00:16:32|     376|         7.3|          .86|     1.1649|        8.234|
|       2| 2019-01-01 00:27:11|  2019-01-01 00:31:38|     267|         5.8|          .66|     1.3034|       8.8989|
|       2| 2019-01-01 00:46:20|  2019-01-01 01:04:54|    1114|       19.71|         2.68|     1.0616|       8.6607|
|       2| 2019-01-01 00:19:06|  2019-01-01 00:39:43|    1237|        19.3|         4.53|     0.9361|      13.1835|
|       2| 2019-01-01 00:12:35|  2019-01-01 00:19:09|     394|         7.8|         1.05|     1.1878|       9.5939|
|       2| 2019-01-01 00:47:55|  2019-01-01 01:00:01|     726|        14

In [18]:
## Write Data Back to S3 using AWS API
currenttime = datetime.datetime.now()

df.write.option('compression', 'snappy') \
    .save( "s3://data-etl-o-target-0/result/green/green_car_data_processed_{date}/".format(date=currenttime.strftime('%Y%m%d%H%M%s')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
#read in our raw dataset
dfz = spark.read.option('header', 'true') \
    .option('mode', 'FAILFAST') \
    .option('columnNameOfCorruptRecord', 'error') \
    .csv('s3://data-etl-o-original-raw/zone/nyc.tlc.taxi_zone_lookup.csv') \

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
dfzcache = dfz.cache()

dfz.count(),dfz.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(265, ['LocationID', 'Borough', 'Zone', 'service_zone'])

In [17]:
dfz.show(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [18]:
dfz.select('Borough').distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+
|      Borough|
+-------------+
|       Queens|
|          EWR|
|      Unknown|
|     Brooklyn|
|Staten Island|
|    Manhattan|
|        Bronx|
+-------------+

In [89]:
dfz.show(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [17]:
dfz.select('Borough').groupBy('Borough').agg({'Borough': 'count'}).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+--------------+
|      Borough|count(Borough)|
+-------------+--------------+
|       Queens|            69|
|          EWR|             1|
|      Unknown|             2|
|     Brooklyn|            61|
|Staten Island|            20|
|    Manhattan|            69|
|        Bronx|            43|
+-------------+--------------+

In [99]:
%%local
conda list

# packages in environment at /opt/conda:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                 conda_forge    conda-forge
_openmp_mutex             4.5                      1_llvm    conda-forge
alabaster                 0.7.12                     py_0    defaults
anaconda-client           1.7.2                    py37_0    defaults
anaconda-navigator        1.9.7                    py37_0    defaults
anaconda-project          0.8.4                      py_0    defaults
asn1crypto                1.3.0                    py37_0    defaults
astroid                   2.3.3                    py37_0    defaults
astropy                   4.0              py37h7b6447c_0    defaults
attrs                     19.3.0                     py_0    defaults
autovizwidget             0.13.1                     py_0    defaults
awscli                    1.18.3                   py37_0    conda-forge
awseditorssparkmonitoringwidget 1.0      

In [100]:
%%help

Magic,Example,Explanation
info,%%info,Outputs session information for the current Livy endpoint.
cleanup,%%cleanup -f,"Deletes all sessions for the current Livy endpoint, including this notebook's session. The force flag is mandatory."
delete,%%delete -f -s 0,Deletes a session by number for the current Livy endpoint. Cannot delete this kernel's session.
logs,%%logs,Outputs the current session's Livy logs.
configure,"%%configure -f{""executorMemory"": ""1000M"", ""executorCores"": 4}",Configure the session creation parameters. The force flag is mandatory if a session has already been created and the session will be dropped and recreated.Look at Livy's POST /sessions Request Body for a list of valid parameters. Parameters must be passed in as a JSON string.
spark,%%spark -o dfdf = spark.read.parquet('...,"Executes spark commands. Parameters: -o VAR_NAME: The Spark dataframe of name VAR_NAME will be available in the %%local Python context as a Pandas dataframe with the same name. -m METHOD: Sample method, either take or sample. -n MAXROWS: The maximum number of rows of a dataframe that will be pulled from Livy to Jupyter. If this number is negative, then the number of rows will be unlimited. -r FRACTION: Fraction used for sampling."
display,%%display -n 100my_spark_df,"Displays Spark dataframe as a beautiful HTML table with horizontal and vertical scroll bars(if necessary). Parameters: -m METHOD: Sample method, either take or sample. -n MAXROWS: The maximum number of rows of a dataframe to display. If this number is negative, then the number of rows will be unlimited. -r FRACTION: Fraction used for sampling."
sql,%%sql -o tables -qSHOW TABLES,"Executes a SQL query against the variable sqlContext (Spark v1.x) or spark (Spark v2.x). Parameters: -o VAR_NAME: The result of the SQL query will be available in the %%local Python context as a Pandas dataframe. -q: The magic will return None instead of the dataframe (no visualization). -m, -n, -r are the same as the %%spark parameters above."
local,%%locala = 1,All the code in subsequent lines will be executed locally. Code must be valid Python code.


In [101]:
tmp = df.select(['VendorID','PULocationID', 'DOLocationID', 'passenger_count', 'lpep_pickup_datetime', 'lpep_dropoff_datetime', 'duration','total_amount','trip_distance','minute_rate','average_speed'])
tmp.show(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------------+------------+---------------+--------------------+---------------------+--------+------------+-------------+-----------+-------------+
|VendorID|PULocationID|DOLocationID|passenger_count|lpep_pickup_datetime|lpep_dropoff_datetime|duration|total_amount|trip_distance|minute_rate|average_speed|
+--------+------------+------------+---------------+--------------------+---------------------+--------+------------+-------------+-----------+-------------+
|       2|          97|          49|              2| 2019-01-01 00:10:16|  2019-01-01 00:16:32|     376|         7.3|          .86|      1.165|        8.234|
|       2|          49|         189|              2| 2019-01-01 00:27:11|  2019-01-01 00:31:38|     267|         5.8|          .66|      1.303|        8.899|
|       2|         189|          17|              2| 2019-01-01 00:46:20|  2019-01-01 01:04:54|    1114|       19.71|         2.68|      1.062|        8.661|
|       2|          82|         258|              1|

In [19]:
df.groupBy('passenger_count').count().orderBy('count', ascending=False).limit(10).show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-------+
|passenger_count|  count|
+---------------+-------+
|              1|4705631|
|              2| 406405|
|           null| 304825|
|              5| 161783|
|              6|  86660|
|              3|  75135|
|              4|  28172|
|              0|  10584|
|              8|     58|
|              7|     36|
+---------------+-------+

In [91]:
toplo = df.groupBy('PULocationID').count().orderBy('count', ascending=False).limit(10)
toplo.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------+
|PULocationID| count|
+------------+------+
|          74|457848|
|          75|368752|
|          41|333623|
|          82|259303|
|           7|244131|
+------------+------+
only showing top 5 rows

In [93]:
toplo2 = df.groupBy('PULocationID','DOLocationID').count().orderBy('count', ascending=False).limit(10)
toplo2.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------+-----+
|PULocationID|DOLocationID|count|
+------------+------------+-----+
|          75|          74|72275|
|           7|           7|67275|
|          74|          75|61992|
|          41|          42|56941|
|          95|          95|52950|
+------------+------------+-----+
only showing top 5 rows

In [21]:
dfz.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows

In [94]:
top_5_loc_names = toplo2.join(dfz, toplo2['PULocationID'].eqNullSafe(dfz['LocationID']),'left') \
            .withColumnRenamed('Borough','FromLocation') \
            .drop('LocationID') \
            .orderBy('count', ascending=False)
top_5_loc_names.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------+-----+------------+-----------------+------------+
|PULocationID|DOLocationID|count|FromLocation|             Zone|service_zone|
+------------+------------+-----+------------+-----------------+------------+
|          75|          74|72275|   Manhattan|East Harlem South|   Boro Zone|
|           7|           7|67275|      Queens|          Astoria|   Boro Zone|
|          74|          75|61992|   Manhattan|East Harlem North|   Boro Zone|
|          41|          42|56941|   Manhattan|   Central Harlem|   Boro Zone|
|          95|          95|52950|      Queens|     Forest Hills|   Boro Zone|
|          82|         129|49679|      Queens|         Elmhurst|   Boro Zone|
|          41|          41|43924|   Manhattan|   Central Harlem|   Boro Zone|
|          74|          41|43754|   Manhattan|East Harlem North|   Boro Zone|
|          74|          74|43477|   Manhattan|East Harlem North|   Boro Zone|
|         129|         129|43268|      Queens|  Jackson Heights|

In [114]:
%%help

Magic,Example,Explanation
info,%%info,Outputs session information for the current Livy endpoint.
cleanup,%%cleanup -f,"Deletes all sessions for the current Livy endpoint, including this notebook's session. The force flag is mandatory."
delete,%%delete -f -s 0,Deletes a session by number for the current Livy endpoint. Cannot delete this kernel's session.
logs,%%logs,Outputs the current session's Livy logs.
configure,"%%configure -f{""executorMemory"": ""1000M"", ""executorCores"": 4}",Configure the session creation parameters. The force flag is mandatory if a session has already been created and the session will be dropped and recreated.Look at Livy's POST /sessions Request Body for a list of valid parameters. Parameters must be passed in as a JSON string.
spark,%%spark -o dfdf = spark.read.parquet('...,"Executes spark commands. Parameters: -o VAR_NAME: The Spark dataframe of name VAR_NAME will be available in the %%local Python context as a Pandas dataframe with the same name. -m METHOD: Sample method, either take or sample. -n MAXROWS: The maximum number of rows of a dataframe that will be pulled from Livy to Jupyter. If this number is negative, then the number of rows will be unlimited. -r FRACTION: Fraction used for sampling."
display,%%display -n 100my_spark_df,"Displays Spark dataframe as a beautiful HTML table with horizontal and vertical scroll bars(if necessary). Parameters: -m METHOD: Sample method, either take or sample. -n MAXROWS: The maximum number of rows of a dataframe to display. If this number is negative, then the number of rows will be unlimited. -r FRACTION: Fraction used for sampling."
sql,%%sql -o tables -qSHOW TABLES,"Executes a SQL query against the variable sqlContext (Spark v1.x) or spark (Spark v2.x). Parameters: -o VAR_NAME: The result of the SQL query will be available in the %%local Python context as a Pandas dataframe. -q: The magic will return None instead of the dataframe (no visualization). -m, -n, -r are the same as the %%spark parameters above."
local,%%locala = 1,All the code in subsequent lines will be executed locally. Code must be valid Python code.


In [23]:
top_5_loc_names.createOrReplaceTempView("top_5_loc_names_views")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
%%sql -o top_5_loc_names -n -1
select 'PULocationID','DOLocationID','count' from top_5_loc_names_views
group by 'PULocationID','DOLocationID'
ORDER BY count Desc

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [21]:
import os
all_files_in_cur_dir = os.listdir('.') 
csvfileslist = list(filter(lambda x:(x[-6:] == '.ipynb'),all_files_in_cur_dir))
csvfileslist

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[]

In [34]:
import os
# csvfileslist = list(os.listdir('.'))
csvfileslist
os.listdir('.')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['var', 'lib64', 'emr', 'home', 'lost+found', 'sbin', 'selinux', 'lib', 'srv', 'device-names.json', 'media', 'tmp', 'mnt', 'run', 'cgroup', 'bin', 'etc', 'dev', 'mnt1', '.autofsck', 'boot', 'opt', 'usr', 'proc', 'sys', 'root', 'local', '.autorelabel']

In [22]:
tmpfile = list(filter(lambda x:(x[-4:] == '.zip'),os.listdir('/tmp')))
tmpfile

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['LoanStats3a.csv.zip', 'taxi_zones.zip']

In [46]:
import zipfile
urllib.request.urlretrieve("https://s3.amazonaws.com/nyc-tlc/misc/taxi_zones.zip", "/tmp/taxi_zones.zip")
with zipfile.ZipFile("/tmp/taxi_zones.zip","r") as zip_ref:
    zip_ref.extractall("/tmp/shape")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
os.listdir('/tmp/shape')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['taxi_zones.prj', 'taxi_zones.dbf', 'taxi_zones.shp', 'taxi_zones.shp.xml', 'taxi_zones.shx', 'taxi_zones.sbn', 'taxi_zones.sbx']

In [54]:
#read in our raw dataset
# shpt1 = sc.binaryFiles('s3://data-etl-o-original-raw/zone/taxi_zones_shape.zip')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
# shpt1.saveAsTextFile('/tmp/taxi_zones_shape_1.zip')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
sc.install_pypi_package("pandas")
sc.install_pypi_package("pyshp")
sc.install_pypi_package("shapely")
sc.install_pypi_package("descartes")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached pandas-1.0.1-cp36-cp36m-manylinux1_x86_64.whl (10.1 MB)
  Using cached python_dateutil-2.8.1-py2.py3-none-any.whl (227 kB)

In [95]:
import pandas as pd
import numpy as np
import urllib.request
import zipfile
import random
import itertools
import math
import shapefile
from shapely.geometry import Polygon
from descartes.patch import PolygonPatch
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('ggplot')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
sf = shapefile.Reader("/tmp/shape/taxi_zones.shp")
sf

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
sf.fields

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('DeletionFlag', 'C', 1, 0), ['OBJECTID', 'N', 9, 0], ['Shape_Leng', 'F', 19, 11], ['Shape_Area', 'F', 19, 11], ['zone', 'C', 254, 0], ['LocationID', 'N', 4, 0], ['borough', 'C', 254, 0]]

In [75]:
sf.fields[1:]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[['OBJECTID', 'N', 9, 0], ['Shape_Leng', 'F', 19, 11], ['Shape_Area', 'F', 19, 11], ['zone', 'C', 254, 0], ['LocationID', 'N', 4, 0], ['borough', 'C', 254, 0]]

In [77]:
fields_name = [field[0] for field in sf.fields[1:]]
fields_name

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['OBJECTID', 'Shape_Leng', 'Shape_Area', 'zone', 'LocationID', 'borough']

In [78]:
shp_dic = dict(zip(fields_name, list(range(len(fields_name)))))
shp_dic

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'OBJECTID': 0, 'Shape_Leng': 1, 'Shape_Area': 2, 'zone': 3, 'LocationID': 4, 'borough': 5}

In [80]:
attributes = sf.records()
for attr in attributes:
    attr

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Record #0: [1, 0.116357453189, 0.0007823067885, 'Newark Airport', 1, 'EWR'], Record #1: [2, 0.43346966679, 0.00486634037837, 'Jamaica Bay', 2, 'Queens'], Record #2: [3, 0.0843411059012, 0.000314414156821, 'Allerton/Pelham Gardens', 3, 'Bronx'], Record #3: [4, 0.0435665270921, 0.000111871946192, 'Alphabet City', 4, 'Manhattan'], Record #4: [5, 0.0921464898574, 0.000497957489363, 'Arden Heights', 5, 'Staten Island'], Record #5: [6, 0.150490542523, 0.000606460984581, 'Arrochar/Fort Wadsworth', 6, 'Staten Island'], Record #6: [7, 0.107417171123, 0.000389787989274, 'Astoria', 7, 'Queens'], Record #7: [8, 0.0275906911574, 2.6587716279e-05, 'Astoria Park', 8, 'Queens'], Record #8: [9, 0.0997840924705, 0.000338443803197, 'Auburndale', 9, 'Queens'], Record #9: [10, 0.0998394794152, 0.000435823818081, 'Baisley Park', 10, 'Queens'], Record #10: [11, 0.0792110389596, 0.00026452053504, 'Bath Beach', 11, 'Brooklyn'], Record #11: [12, 0.0366613013579, 4.15116236727e-05, 'Battery Park', 12, 'Manhatta

In [93]:
shp_attr = [dict(zip(fields_name, attr)) for attr in attributes]
for sattr in shp_attr:
    sattr

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'OBJECTID': 1, 'Shape_Leng': 0.116357453189, 'Shape_Area': 0.0007823067885, 'zone': 'Newark Airport', 'LocationID': 1, 'borough': 'EWR'}
{'OBJECTID': 2, 'Shape_Leng': 0.43346966679, 'Shape_Area': 0.00486634037837, 'zone': 'Jamaica Bay', 'LocationID': 2, 'borough': 'Queens'}
{'OBJECTID': 3, 'Shape_Leng': 0.0843411059012, 'Shape_Area': 0.000314414156821, 'zone': 'Allerton/Pelham Gardens', 'LocationID': 3, 'borough': 'Bronx'}
{'OBJECTID': 4, 'Shape_Leng': 0.0435665270921, 'Shape_Area': 0.000111871946192, 'zone': 'Alphabet City', 'LocationID': 4, 'borough': 'Manhattan'}
{'OBJECTID': 5, 'Shape_Leng': 0.0921464898574, 'Shape_Area': 0.000497957489363, 'zone': 'Arden Heights', 'LocationID': 5, 'borough': 'Staten Island'}
{'OBJECTID': 6, 'Shape_Leng': 0.150490542523, 'Shape_Area': 0.000606460984581, 'zone': 'Arrochar/Fort Wadsworth', 'LocationID': 6, 'borough': 'Staten Island'}
{'OBJECTID': 7, 'Shape_Leng': 0.107417171123, 'Shape_Area': 0.000389787989274, 'zone': 'Astoria', 'LocationID': 7, 'b

In [90]:
for sr in sf.shapeRecords():
    sr.shape.bbox[0:4], sr.record

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(array('d', [931553.4909607167, 183788.04973023868, 941810.0086498931, 197256.2108251324]), Record #0: [1, 0.116357453189, 0.0007823067885, 'Newark Airport', 1, 'EWR'])
(array('d', [1018052.3591919094, 151230.39099124074, 1049019.8063964993, 172477.57360841334]), Record #1: [2, 0.43346966679, 0.00486634037837, 'Jamaica Bay', 2, 'Queens'])
(array('d', [1022540.6604834348, 251648.65237039328, 1031732.1639056653, 257811.3693266213]), Record #2: [3, 0.0843411059012, 0.000314414156821, 'Allerton/Pelham Gardens', 3, 'Bronx'])
(array('d', [988733.885187909, 201170.6183192879, 992114.1539525241, 205029.46254441142]), Record #3: [4, 0.0435665270921, 0.000111871946192, 'Alphabet City', 4, 'Manhattan'])
(array('d', [927766.5390208364, 134554.84833614528, 936499.88067846, 145354.2355352044]), Record #4: [5, 0.0921464898574, 0.000497957489363, 'Arden Heights', 5, 'Staten Island'])
(array('d', [958451.7027633935, 153868.42604996264, 969746.3320312649, 161198.91807557642]), Record #5: [6, 0.150490542

In [ ]:
def get_lat_lon(sf,shp_dic):
    content = []
    for sr in sf.shapeRecords():
        shape = sr.shape
        rec = sr.record
        loc_id = rec[shp_dic['LocationID']]
        x = (shape.bbox[0]+shape.bbox[2])/2
        y = (shape.bbox[1]+shape.bbox[3])/2
        content.append((loc_id, x, y))
    return pd.DataFrame(content, columns=["LocationID", "longitude", "latitude"])

VBox()

In [ ]:
sf = shapefile.Reader("/tmp/shape/taxi_zones.shp")
fields_name = [field[0] for field in sf.fields[1:]]
shp_dic = dict(zip(fields_name, list(range(len(fields_name)))))
attributes = sf.records()
shp_attr = [dict(zip(fields_name, attr)) for attr in attributes]

df_loc = pd.DataFrame(shp_attr).join(get_lat_lon(sf,shp_dic).set_index("LocationID"), on="LocationID")

df_loc.head()

In [38]:
spark_dfl = spark.createDataFrame(df_loc)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
spark_dfl.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------------+----------------+--------------------+----------+-------------+------------------+------------------+
|OBJECTID|     Shape_Leng|      Shape_Area|                zone|LocationID|      borough|         longitude|          latitude|
+--------+---------------+----------------+--------------------+----------+-------------+------------------+------------------+
|       1| 0.116357453189|  7.823067885E-4|      Newark Airport|         1|          EWR| 936681.7498053049|190522.13027768553|
|       2|  0.43346966679|0.00486634037837|         Jamaica Bay|         2|       Queens|1033536.0827942044|161853.98229982704|
|       3|0.0843411059012|3.14414156821E-4|Allerton/Pelham G...|         3|        Bronx|  1027136.41219455|254730.01084850729|
|       4|0.0435665270921|1.11871946192E-4|       Alphabet City|         4|    Manhattan| 990424.0195702165|203100.04043184966|
|       5|0.0921464898574|4.97957489363E-4|       Arden Heights|         5|Staten Island| 932133.2098496

In [78]:
def draw_zone_map(ax, sf, heat={}, text=[], arrows=[]):
    continent = [235/256, 151/256, 78/256]
    ocean = (89/256, 171/256, 227/256)
    theta = np.linspace(0, 2*np.pi, len(text)+1).tolist()
    ax.set_facecolor(ocean)
    
    # colorbar
    if len(heat) != 0:
        norm = mpl.colors.Normalize(vmin=fmin(heat.values()),vmax=max(heat.values())) #norm = mpl.colors.LogNorm(vmin=1,vmax=max(heat))
        cm=plt.get_cmap('Reds')
        sm = plt.cm.ScalarMappable(cmap=cm, norm=norm)
        sm.set_array([])
        plt.colorbar(sm, ticks=np.linspace(min(heat.values()),max(heat.values()),8),
                     boundaries=np.arange(min(heat.values())-10,max(heat.values())+10,.1))
    
    for sr in sf.shapeRecords():
        shape = sr.shape
        rec = sr.record
        loc_id = rec[shp_dic['LocationID']]
        zone = rec[shp_dic['zone']]
        
        if len(heat) == 0:
            col = continent
        else:
            if loc_id not in heat:
                R,G,B,A = cm(norm(0))
            else:
                R,G,B,A = cm(norm(heat[loc_id]))
            col = [R,G,B]

        # check number of parts (could use MultiPolygon class of shapely?)
        nparts = len(shape.parts) # total parts
        if nparts == 1:
            polygon = Polygon(shape.points)
            patch = PolygonPatch(polygon, facecolor=col, alpha=1.0, zorder=2)
            ax.add_patch(patch)
        else: # loop over parts of each shape, plot separately
            for ip in range(nparts): # loop over parts, plot separately
                i0 = shape.parts[ip]
                if ip < nparts-1:
                    i1 = shape.parts[ip+1]-1
                else:
                    i1 = len(shape.points)

                polygon = Polygon(shape.points[i0:i1+1])
                patch = PolygonPatch(polygon, facecolor=col, alpha=1.0, zorder=2)
                ax.add_patch(patch)
        
        x = (shape.bbox[0]+shape.bbox[2])/2
        y = (shape.bbox[1]+shape.bbox[3])/2
        if (len(text) == 0 and rec[shp_dic['Shape_Area']] > 0.0001):
            plt.text(x, y, str(loc_id), horizontalalignment='center', verticalalignment='center')            
        elif len(text) != 0 and loc_id in text:
            #plt.text(x+0.01, y-0.01, str(loc_id), fontsize=12, color="white", bbox=dict(facecolor='black', alpha=0.5))
            eta_x = 0.05*np.cos(theta[text.index(loc_id)])
            eta_y = 0.05*np.sin(theta[text.index(loc_id)])
            ax.annotate("[{}] {}".format(loc_id, zone), xy=(x, y), xytext=(x+eta_x, y+eta_y),
                        bbox=dict(facecolor='black', alpha=0.5), color="white", fontsize=12,
                        arrowprops=dict(facecolor='black', width=3, shrink=0.05))
    if len(arrows)!=0:
        for arr in arrows:
            ax.annotate('', xy = arr['dest'], xytext = arr['src'], size = arr['cnt'],
                    arrowprops=dict(arrowstyle="fancy", fc="0.6", ec="none"))
    
    # display
    limits = get_boundaries(sf)
    plt.xlim(limits[0], limits[1])
    plt.ylim(limits[2], limits[3])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
# %matplotlib inline

def draw_region_map(ax, sf, heat={}):
    continent = [235/256, 151/256, 78/256]
    ocean = (89/256, 171/256, 227/256)    
    
    reg_list={'Staten Island':1, 'Queens':2, 'Bronx':3, 'Manhattan':4, 'EWR':5, 'Brooklyn':6}
    reg_x = {'Staten Island':[], 'Queens':[], 'Bronx':[], 'Manhattan':[], 'EWR':[], 'Brooklyn':[]}
    reg_y = {'Staten Island':[], 'Queens':[], 'Bronx':[], 'Manhattan':[], 'EWR':[], 'Brooklyn':[]}
    
    # colorbar
    if len(heat) != 0:
        norm = mpl.colors.Normalize(vmin=math.sqrt(min(heat.values())), vmax=math.sqrt(max(heat.values()))) #norm = mpl.colors.LogNorm(vmin=1,vmax=max(heat))
        cm=plt.get_cmap('Reds')
        #sm = plt.cm.ScalarMappable(cmap=cm, norm=norm)
        #sm.set_array([])
        #plt.colorbar(sm, ticks=np.linspace(min(heat.values()),max(heat.values()),8), \
        #             boundaries=np.arange(min(heat.values())-10,max(heat.values())+10,.1))
    
    ax.set_facecolor(ocean)
    for sr in sf.shapeRecords():
        shape = sr.shape
        rec = sr.record
        reg_name = rec[shp_dic['borough']]
        
        if len(heat) == 0:
            norm = mpl.colors.Normalize(vmin=1,vmax=6) #norm = mpl.colors.LogNorm(vmin=1,vmax=max(heat))
            cm=plt.get_cmap('Pastel1')
            R,G,B,A = cm(norm(reg_list[reg_name]))
            col = [R,G,B]
        else:
            R,G,B,A = cm(norm(math.sqrt(heat[reg_name])))
            col = [R,G,B]
            
        # check number of parts (could use MultiPolygon class of shapely?)
        nparts = len(shape.parts) # total parts
        if nparts == 1:
            polygon = Polygon(shape.points)
            patch = PolygonPatch(polygon, facecolor=col, alpha=1.0, zorder=2)
            ax.add_patch(patch)
        else: # loop over parts of each shape, plot separately
            for ip in range(nparts): # loop over parts, plot separately
                i0 = shape.parts[ip]
                if ip < nparts-1:
                    i1 = shape.parts[ip+1]-1
                else:
                    i1 = len(shape.points)

                polygon = Polygon(shape.points[i0:i1+1])
                patch = PolygonPatch(polygon, facecolor=col, alpha=1.0, zorder=2)
                ax.add_patch(patch)
                
        reg_x[reg_name].append((shape.bbox[0]+shape.bbox[2])/2)
        reg_y[reg_name].append((shape.bbox[1]+shape.bbox[3])/2)
        
    for k in reg_list:
        if len(heat)==0:
            plt.text(np.mean(reg_x[k]), np.mean(reg_y[k]), k, horizontalalignment='center', verticalalignment='center',
                        bbox=dict(facecolor='black', alpha=0.5), color="white", fontsize=12)     
        else:
            plt.text(np.mean(reg_x[k]), np.mean(reg_y[k]), "{}\n({}K)".format(k, heat[k]/1000), horizontalalignment='center', 
                     verticalalignment='center',bbox=dict(facecolor='black', alpha=0.5), color="white", fontsize=12)       

    # display
    limits = get_boundaries(sf)
    plt.xlim(limits[0], limits[1])
    plt.ylim(limits[2], limits[3])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [80]:
# %matplotlib inline

def get_boundaries(sf):
    lat, lon = [], []
    for shape in list(sf.iterShapes()):
        lat.extend([shape.bbox[0], shape.bbox[2]])
        lon.extend([shape.bbox[1], shape.bbox[3]])

    margin = 0.01 # buffer to add to the range
    lat_min = min(lat) - margin
    lat_max = max(lat) + margin
    lon_min = min(lon) - margin
    lon_max = max(lon) + margin

    return lat_min, lat_max, lon_min, lon_max

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [81]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15,8))
ax = plt.subplot(1, 2, 1)
ax.set_title("Borough Area in NYC")
draw_region_map(ax, sf)

ax = plt.subplot(1, 2, 2)
ax.set_title("Zones in NYC")
draw_zone_map(ax, sf)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error occurred while calling z:org.apache.spark.sql.functions.min. Trace:
py4j.Py4JException: Method min([class java.util.ArrayList]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:339)
	at py4j.Gateway.invoke(Gateway.java:276)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:238)
	at java.lang.Thread.run(Thread.java:748)


Traceback (most recent call last):
  File "<stdin>", line 65, in draw_region_map
  File "<stdin>", line 10, in get_boundaries
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/functions.py", line 44, in _
    jc = getattr(sc._jvm.functions, name)(col._jc if isinstance(col, Column) else col)
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gatewa